In [10]:
import os
import json
from pathlib import Path
from langchain.text_splitter import TokenTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, Qdrant
from langchain.document_loaders import TextLoader

In [11]:
keys_file = Path(".").resolve().parent / "keys.json"
print(keys_file)
with open(keys_file) as f:
    keys = json.load(f)
os.environ["OPENAI_API_KEY"] = keys["OPENAI_API_KEY"]

D:\00Repos\GPT-Swarm\keys.json


In [12]:
embeddings = OpenAIEmbeddings()

In [13]:
persist_directory = Path("D:\\00Repos\\GPT-Swarm\\tmp\\swarm\\shared_memory")
vectordb = Chroma(persist_directory=str(persist_directory), embedding_function=embeddings)
retriever_chroma = vectordb.as_retriever(search_type="mmr")

Using embedded DuckDB with persistence: data will be stored in: D:\00Repos\GPT-Swarm\tmp\swarm\shared_memory


In [14]:
vectordb._collection.count()

148

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chains.question_answering import load_qa_chain

In [16]:
model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0) # 'ada' 'gpt-3.5-turbo' 'gpt-4',
qa_chain = load_qa_chain(model, chain_type="stuff")
qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=retriever_chroma)

In [19]:
"""
    - Generate a comprehensive description of the startup. Describe the value proposition, the product, USP and business model of a startup.
    - Find any mentions of the startup in the news, social media, etc.
    - Find top companies and startups in this field. Find out their locations, raised funding, value proposition, differentiation, etc.
    - Find top investors in this field. Including specific details like 'company AAA (link) invested in company BBB (link) $XX in year YYYY'
    - Describe the market size, growth rate and trends of this field.
    - Main problems and challenges of the field. Create an extensive list of problems. What can stop the field from growing? What can stop the company from succeeding?
    - Briefly describe the technology for the non-tech audience. Include links to the main articles in the field.
    - Who are the main influencers in this field?
    - What questions should we ask the startup to make a more informed decision? Avoid generic and obvious questions and focus on field/domain specific questions that can uncover problems with this specific startup.
"""

context = (
    "A new startup just send us their pitch. Find if the startup is worth investing in. The startup is called Brainamics and it is in the space of brain computer interfaces."
)

questions = [
    "Generate a comprehensive description of the startup. Describe the value proposition, the product, USP and business model of a startup.",
    "Find any mentions of the startup in the news, social media, etc.",
    "Find top companies and startups in this field. Find out their locations, raised funding, value proposition, differentiation, etc.",
    "Find top investors in this field. Including specific details like 'company AAA (link) invested in company BBB (link) $XX in year YYYY'",
    "Describe the market size, growth rate and trends of this field.",
    "Main problems and challenges of the field. Create an extensive list of problems. What can stop the field from growing? What can stop the company from succeeding?",
    "Briefly describe the technology for the non-tech audience. Include links to the main articles in the field.",
    "Who are the main influencers in this field?",
    "What questions should we ask the startup to make a more informed decision? Avoid generic and obvious questions and focus on field/domain specific questions that can uncover problems with this specific startup."
]

for question in questions:  
    result = qa.run(context+question+"Be very brief and concise. Focus on the essential information and provide https links to the sources.")
    print(f"=====================================================================================================")
    print(f"=====================================================================================================")
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result} \n")

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


-> **Question**: Generate a comprehensive description of the startup. Describe the value proposition, the product, USP and business model of a startup. 

**Answer**: Brainamics is a startup in the brain computer interface space, specializing in non-invasive BCIs that allow people with disabilities to communicate and control their environment more easily. Their unique selling proposition is that their BCIs do not require surgery or implants. Instead, they use EEG sensors to detect brain activity. Their value proposition addresses a significant need in the market. 

Their product and business model are not specified in the information provided. 

Sources: 
- Comprehensive description: information found in original context provided to me.
- Value Proposition and USP: information found in original context provided to me.
- Product and Business Model: none provided in original context. 

-> **Question**: Find any mentions of the startup in the news, social media, etc. 

**Answer**: As an AI